<a href="https://colab.research.google.com/github/AverYuchen/DeepLearning/blob/main/CNN_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNs in PyTorch

In this assignment, you'll implement some Convolutional Neural Networks (CNNs) in PyTorch.

## Setting up

We'll start by importing the following:
- [`torch`](https://pytorch.org/docs/stable/torch.html) - the core PyTorch library.
- [`torch.nn`](https://pytorch.org/docs/stable/nn.html) - a module containing building blocks for NNs such as linear layers, convolutional layers, and so on.
- [`torch.nn.functional`](https://pytorch.org/docs/stable/nn.functional.html) - a module containing activation functions, loss functions, and so on.
- [`torch.optim`](https://pytorch.org/docs/stable/optim.html) - a module containing optimizers which update the parameters of a NN.
- [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) - Can be used to batch data together and iterate over batches, shuffle data, and parallelize the training process to speed it up.
- [`MNIST`](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html) in [`torchvision.datasets`](https://pytorch.org/vision/stable/datasets.html) - The [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) is a collection of images of handwritten digits.
- [`ToTensor`](https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html#torchvision.transforms.ToTensor) in [`torchvision.transforms`](https://pytorch.org/vision/0.9/transforms.html) - Converts PIL images or NumPy arrays to PyTorch tensors.

In [1]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms.v2 import ToTensor
from torchvision import datasets, transforms

## Data

Let's define a transformation for the [CIFAR10 dataset](https://en.wikipedia.org/wiki/CIFAR-10).

We'll first cast the images to PyTorch tensors using [`transforms.ToTensor()`](https://pytorch.org/vision/master/generated/torchvision.transforms.ToTensor.html). These tensors are automatically normalized such that their values are between 0 and 1.

Then, we'll re-normalize the pixel values with [`transforms.Normalize()`](https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html) to conform approximately to a standard normal distribution, assuming the mean and standard deviation of any channel of the returned tensor to be 0.5. This is not an unreasonable assumption. It's also a fairly standard thing to do to squash inputs to be in (or close to) the range [-1,1], which is where neural networks work best in terms of converging when performing optimization.

In [2]:
# CIFAR-10 transform - three channels, normalize with 3 means and 3 SDs
cifar_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# MNIST transform - single channel, so only 1 mean and 1 SD
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1308,), (0.3016,))
])

In [3]:
train_mnist = MNIST(
    root='./data',
    train=True,
    download=True,
    transform=mnist_transform
)
test_mnist = MNIST(
    root='./data',
    train=False,
    download=True,
    transform=mnist_transform
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15247914.65it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 520089.88it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3564598.48it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2189212.68it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Let's load up the CIFAR-10 dataset. You can specify the split you want using `train=True|False`. `root` is the directory where the dataset will be saved. You can also directly apply the transform from the previous cell by specifying `transform`.

In [4]:
from torch.utils.data import random_split
# CIFAR10 data
train_dataset = CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=cifar_transform
)
val_dataset = CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=cifar_transform
)
'''
test_dataset = CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=cifar_transform
)


val_size = 5000
test_size = len(test_dataset) - val_size
val_dataset, test_dataset = random_split(test_dataset, [val_size, test_size])
'''

100%|██████████| 170498071/170498071 [00:05<00:00, 30121002.86it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


"\ntest_dataset = CIFAR10(\n    root='./data',\n    train=False,\n    download=True,\n    transform=cifar_transform\n)\n\n\nval_size = 5000\ntest_size = len(test_dataset) - val_size\nval_dataset, test_dataset = random_split(test_dataset, [val_size, test_size])\n"

Let's define `DataLoader` objects for the CIFAR10 data now.

We'll use a (mini) batch size of 32. It's common to use powers of 2 in deep learning because it's more efficient to handle such numbers on hardware.

We'll define separate `DataLoader` objects to handle our training and test splits to avoid data leakage (training on the test set or testing on the train set).

We'll also have the `DataLoader` objects shuffle our data whenever we iterate over them (`shuffle=True`). Shuffling data at each epoch is beneficial in that the model won't be optimized in a way that depends on a specific ordering of the data.

Finally, we'll parallelize the loading of the data using 4 CPU processes to load data (`num_workers=4`).

In [5]:
# dataloaders for MNIST
batch_size = 32

train_loader_mnist = DataLoader(
    train_mnist,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

val_loader_mnist = DataLoader(
    test_mnist,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
# dataloaders
batch_size = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

'''
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)
'''

'\ntest_loader = DataLoader(\n    test_dataset,\n    batch_size=batch_size,\n    shuffle=True,\n    num_workers=4\n)\n'

## Defining and training CNNs

We'll define `criterion` to be [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), a common loss function used to train classification models.

We'll also define a Stochastic Gradient Descent optmizizer ([`optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)), which will optimize the parameters of `net`. We'll set two hyperparameters manually: the learning rate (`lr=0.001`) and the momentum (`momentum=0.9`).

In [7]:
# Loss fuction and optimizer
def get_crit_and_opt(net):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    return criterion, optimizer

Let's see how [LeNet5](https://ieeexplore.ieee.org/document/726791) (Lecun et al. 1998) is implemented. The architecture looks something like this:

![](https://drive.google.com/uc?export=view&id=1PwYfmSXqBnosIQi-ewrr03Ibd_lmRtea)

LeNet5 is compatible with the MNIST dataset. Let's see how to implement the architecture in PyTorch:

In [8]:
class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 6 input channels to 16 output channels with 5x5 convolution
        self.conv2 = nn.Conv2d(6, 16, 5)
        # affine operations: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 16 channels each of size 5x5 to 1x120 vector
        self.fc2 = nn.Linear(120, 84) # 1x120 vector to 1x84 vector
        self.fc3 = nn.Linear(84, 10) # 1x84 vector to 1x10 vector

    def forward(self, x):
        # average pooling over a 2x2 window
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.sigmoid(self.fc1(x)) # linear + sig activation
        x = F.sigmoid(self.fc2(x)) # linear + sig activation
        x = self.fc3(x) # linear
        return x

In general, a PyTorch neural network definition must:
- subclass [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
- call `super().__init__()` in the constructor (`__init__()`) method
- define the trainable parameters/layers (convolutions, linears, poolings, etc.) in the constructor
- define what should happen to the inputs in the `forward()` method

In [9]:
# Create your own model for the MNIST data here [20 pts]:
class CNN_Network1(nn.Module):

  def __init__(self):
    super(CNN_Network1, self).__init__()
    self.conv1 = nn.Conv2d(1, 8, 5)
    self.conv2 = nn.Conv2d(8, 22, 5)
    self.fc1 = nn.Linear(22*4*4, 125)
    self.fc2 = nn.Linear(125, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2))
    # If the size is a square, you can specify with a single number
    x = F.avg_pool2d(F.sigmoid(self.conv2(x)), 2)
    x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
    x = F.sigmoid(self.fc1(x)) # linear + sig activation
    x = F.sigmoid(self.fc2(x)) # linear + sig activation
    x = self.fc3(x) # linear + sig activation
    return x

In [10]:
# Create your own model for the CIFAR10 data here [20 pts]:
class CNN_Network2(nn.Module):

  def __init__(self):
    super(CNN_Network2, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 36, 5)
    self.fc1 = nn.Linear(36 * 5 * 5, 180)
    self.fc2 = nn.Linear(180, 120)
    self.fc3 = nn.Linear(120, 84)
    self.fc4 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 36 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

Below is a useful object for tracking losses/performance during training and dev.

In [11]:
class AverageMeter(object):

    """Computes and stores an average and current value."""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

We'll define an accuracy metric that flexibly computes top-k accuracies.

In [12]:
def error_rate(output, target, topk=(1,)):

    """Computes the top-k error rate for the specified values of k."""

    maxk = max(topk) # largest k we'll need to work with
    batch_size = target.size(0) # determine batch size

    # get maxk best predictions for each item in the batch, both values and indices
    _, pred = output.topk(maxk, 1, True, True)

    # reshape predictions and targets and compare them element-wise
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk: # for each top-k accuracy we want

        # num correct
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        # num incorrect
        wrong_k = batch_size - correct_k
        # as a percentage
        res.append(wrong_k.mul_(100.0 / batch_size))

    return res

If you connect to a runtime with a T4 available, this line will ensure computations that can be done on the T4 are done there.

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

The training function below takes the training set's `DataLoader`, the model we are training, the loss function we are using, and the optimizer for this model.

It then trains the model on the data for 1 epoch.

In [14]:
# training function - 1 epoch
def train(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    epochs,
    print_freq = 100,
    verbose = True
):

    # track average and worst losses
    losses = AverageMeter()

    # set training mode
    model.train()

    # iterate over data - automatically shuffled
    for i, (images, labels) in enumerate(train_loader):

        # put batch of image tensors on GPU
        images = images.to(device)
        # put batch of label tensors on GPU
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # model output
        outputs = model(images)

        # loss computation
        loss = criterion(outputs, labels)

        # back propagation
        loss.backward()

        # update model parameters
        optimizer.step()

        # update meter with the value of the loss once for each item in the batch
        losses.update(loss.item(), images.size(0))

        # logging during epoch
        if i % print_freq == 0 and verbose == True:
            print(
                f'Epoch: [{epoch+1}/{epochs}][{i:4}/{len(train_loader)}]\t'
                f'Loss: {losses.val:.4f} ({losses.avg:.4f} on avg)'
            )

    # log again at end of epoch
    print(f'\n* Epoch: [{epoch+1}/{epochs}]\tTrain loss: {losses.avg:.3f}\n')

    return losses.avg

In [15]:
# val function
def validate(
    val_loader,
    model,
    criterion,
    epoch,
    epochs,
    print_freq = 100,
    verbose = True
):

    # track average and worst losses and batch-wise top-1 and top-5 accuracies
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # set evaluation mode
    model.eval()

    # iterate over data - automatically shuffled
    for i, (images, labels) in enumerate(val_loader):

        # put batch of image tensors on GPU
        images = images.to(device)
        # put batch of label tensors on GPU
        labels = labels.to(device)

        # model output
        output = model(images)

        # loss computation
        loss = criterion(output, labels)

        # top-1 and top-5 accuracy on this batch
        err1, err5, = error_rate(output.data, labels, topk=(1, 5))

        # update meters with the value of the loss once for each item in the batch
        losses.update(loss.item(), images.size(0))
        # update meters with top-1 and top-5 accuracy on this batch once for each item in the batch
        top1.update(err1.item(), images.size(0))
        top5.update(err5.item(), images.size(0))

        # logging during epoch
        if i % print_freq == 0 and verbose == True:
            print(
                f'Test (on val set): [{epoch+1}/{epochs}][{i:4}/{len(val_loader)}]\t'
                f'Loss: {losses.val:.4f} ({losses.avg:.4f} on avg)\t'
                f'Top-1 err: {top1.val:.4f} ({top1.avg:.4f} on avg)\t'
                f'Top-5 err: {top5.val:.4f} ({top5.avg:.4f} on avg)'
            )

    # logging for end of epoch
    print(
        f'\n* Epoch: [{epoch+1}/{epochs}]\t'
        f'Test loss: {losses.avg:.3f}\t'
        f'Top-1 err: {top1.avg:.3f}\t'
        f'Top-5 err: {top5.avg:.3f}\n'
    )

    # avergae top-1 and top-5 accuracies batch-wise, and average loss batch-wise
    return output, labels, top1.avg, top5.avg, losses.avg

In [16]:
# best error rates so far
best_err1 = 100
best_err5 = 100

In [17]:
# Test MNIST Model
if __name__ == '__main__':

    # select a model to train here
    model_mnist = CNN_Network1()

    # move to GPU
    model_mnist.to(device)

    # select number of epochs
    epochs = 3

    # get criterion and optimizer
    criterion, optimizer = get_crit_and_opt(model_mnist)

    # epoch loop
    for epoch in range(0, epochs):

        # train for one epoch
        train_loss_mnist = train(
          train_loader_mnist,
          model_mnist,
          criterion,
          optimizer,
          epoch,
          epochs
        )

        # evaluate on validation set
        predictions_mnist, labels_mnist, err1_mnist, err5_mnist, val_loss_mnist = validate(
          val_loader_mnist,
          model_mnist,
          criterion,
          epoch,
          epochs
        )

        # remember best prec@1 and save checkpoint
        is_best = err1_mnist <= best_err1
        best_err1 = min(err1_mnist, best_err1)
        if is_best:
            best_err5 = err5_mnist

        print('Current best error rate (top-1 and top-5 error):', best_err1, best_err5, '\n')
    print('Best error rate (top-1 and top-5 error):', best_err1, best_err5)

Epoch: [1/3][   0/1875]	Loss: 2.2458 (2.2458 on avg)
Epoch: [1/3][ 100/1875]	Loss: 2.2989 (2.3126 on avg)
Epoch: [1/3][ 200/1875]	Loss: 2.3237 (2.3080 on avg)
Epoch: [1/3][ 300/1875]	Loss: 2.2926 (2.3058 on avg)
Epoch: [1/3][ 400/1875]	Loss: 2.3388 (2.3052 on avg)
Epoch: [1/3][ 500/1875]	Loss: 2.2961 (2.3047 on avg)
Epoch: [1/3][ 600/1875]	Loss: 2.2962 (2.3043 on avg)
Epoch: [1/3][ 700/1875]	Loss: 2.3058 (2.3044 on avg)
Epoch: [1/3][ 800/1875]	Loss: 2.3046 (2.3044 on avg)
Epoch: [1/3][ 900/1875]	Loss: 2.2957 (2.3042 on avg)
Epoch: [1/3][1000/1875]	Loss: 2.2757 (2.3039 on avg)
Epoch: [1/3][1100/1875]	Loss: 2.2916 (2.3039 on avg)
Epoch: [1/3][1200/1875]	Loss: 2.3082 (2.3039 on avg)
Epoch: [1/3][1300/1875]	Loss: 2.3257 (2.3039 on avg)
Epoch: [1/3][1400/1875]	Loss: 2.2965 (2.3039 on avg)
Epoch: [1/3][1500/1875]	Loss: 2.2888 (2.3038 on avg)
Epoch: [1/3][1600/1875]	Loss: 2.3140 (2.3038 on avg)
Epoch: [1/3][1700/1875]	Loss: 2.2791 (2.3037 on avg)
Epoch: [1/3][1800/1875]	Loss: 2.3107 (2.3037 o

In [18]:
# Run the main function.
if __name__ == '__main__':

    # select a model to train here
    model = CNN_Network2()

    # move to GPU
    model.to(device)

    # select number of epochs
    epochs = 3

    # get criterion and optimizer
    criterion, optimizer = get_crit_and_opt(model)

    # epoch loop
    for epoch in range(0, epochs):

        # train for one epoch
        train_loss = train(
          train_loader,
          model,
          criterion,
          optimizer,
          epoch,
          epochs
        )

        # evaluate on validation set
        predictions, labels, err1, err5, val_loss = validate(
          val_loader,
          model,
          criterion,
          epoch,
          epochs
        )

        # remember best prec@1 and save checkpoint
        is_best = err1 <= best_err1
        best_err1 = min(err1, best_err1)
        if is_best:
            best_err5 = err5

        print('Current best error rate (top-1 and top-5 error):', best_err1, best_err5, '\n')
    print('Best error rate (top-1 and top-5 error):', best_err1, best_err5)

Epoch: [1/3][   0/1563]	Loss: 2.3020 (2.3020 on avg)
Epoch: [1/3][ 100/1563]	Loss: 2.3235 (2.3052 on avg)
Epoch: [1/3][ 200/1563]	Loss: 2.2983 (2.3047 on avg)
Epoch: [1/3][ 300/1563]	Loss: 2.3041 (2.3043 on avg)
Epoch: [1/3][ 400/1563]	Loss: 2.2924 (2.3039 on avg)
Epoch: [1/3][ 500/1563]	Loss: 2.3004 (2.3036 on avg)
Epoch: [1/3][ 600/1563]	Loss: 2.3028 (2.3033 on avg)
Epoch: [1/3][ 700/1563]	Loss: 2.2956 (2.3030 on avg)
Epoch: [1/3][ 800/1563]	Loss: 2.2981 (2.3029 on avg)
Epoch: [1/3][ 900/1563]	Loss: 2.3106 (2.3027 on avg)
Epoch: [1/3][1000/1563]	Loss: 2.2987 (2.3025 on avg)
Epoch: [1/3][1100/1563]	Loss: 2.3058 (2.3022 on avg)
Epoch: [1/3][1200/1563]	Loss: 2.3033 (2.3019 on avg)
Epoch: [1/3][1300/1563]	Loss: 2.2944 (2.3016 on avg)
Epoch: [1/3][1400/1563]	Loss: 2.3010 (2.3014 on avg)
Epoch: [1/3][1500/1563]	Loss: 2.2989 (2.3011 on avg)

* Epoch: [1/3]	Train loss: 2.301

Test (on val set): [1/3][   0/313]	Loss: 2.2977 (2.2977 on avg)	Top-1 err: 93.7500 (93.7500 on avg)	Top-5 err: 37.500

Create a SciKit-Learn [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) for one of your models.

This report will show accuracy, classwise precision, recall, and F1, as well as averaged metrics over the classes.

In [23]:
# Create a classification report for one model [10 pts]
from sklearn.metrics import classification_report

# get the true classes and model predictions for the test set for one model
y_true = []
y_pred = []
target_names = val_dataset.classes
_, predicted = torch.max(predictions, 1)  # Get the class with the highest score

labelling = list(range(10))

# Move predictions and labels to CPU, then convert to NumPy arrays
y_true.extend(labels.cpu().numpy())
y_pred.extend(predicted.cpu().numpy())

#target_names = string names of the classes
classification_report(y_true, y_pred, labels = labelling, target_names=target_names, zero_division=1)
print(classification_report(y_true, y_pred, labels = labelling,target_names=target_names, zero_division=1))

              precision    recall  f1-score   support

    airplane       0.67      1.00      0.80         2
  automobile       1.00      0.00      0.00         3
        bird       1.00      1.00      1.00         0
         cat       1.00      0.00      0.00         3
        deer       0.00      1.00      0.00         0
         dog       1.00      0.00      0.00         3
        frog       0.67      1.00      0.80         2
       horse       0.00      1.00      0.00         0
        ship       1.00      1.00      1.00         1
       truck       0.29      1.00      0.44         2

    accuracy                           0.44        16
   macro avg       0.66      0.70      0.40        16
weighted avg       0.83      0.44      0.32        16

